In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import numpy as np

In [2]:
nesting_start = 3
efficient = True
nesting_list = [2**i for i in range(nesting_start, 12)] # 8, 16, 32, 64, 128, 256, 512, 1024, 2048
num_classes = 10 # mnist = 10
model_path = "/home/iliam/dl-project/model/final_weights.pt"  # Update with your actual path

## Loading and checkind datasets

In [3]:
import torchvision
mnist_test = torchvision.datasets.MNIST('./mnist', download=True, train=False, transform=torchvision.transforms.ToTensor())
cifar10_test = torchvision.datasets.CIFAR10('./cifar10', download=True, train=False, transform=torchvision.transforms.ToTensor())

In [4]:
print(len(mnist_test))
input_image, target_class = mnist_test[0]
print(target_class)
input_image.shape

10000
7


torch.Size([1, 28, 28])

In [5]:
print(len(cifar10_test))
input_image, target_clasee = cifar10_test[0]
print(target_class)
input_image.shape

10000
7


torch.Size([3, 32, 32])

# Applying model + MRL to dataset

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
import numpy as np
from tqdm import tqdm
from datasets import load_from_disk, Dataset
from PIL import Image
from mrl_loader import MRLLoader

model = MRLLoader(weights_path=model_path, num_classes=10)

/home/iliam/MRL-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initializing resnet50 model...


/home/iliam/MRL-env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/iliam/MRL-env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Applying BlurPool...
Loading weights from /home/iliam/dl-project/model/final_weights.pt...
Error loading weights: Error(s) in loading state_dict for ResNet:
	size mismatch for fc.nesting_classifier_0.weight: copying a param with shape torch.Size([1000, 2048]) from checkpoint, the shape in current model is torch.Size([10, 2048]).
	size mismatch for fc.nesting_classifier_0.bias: copying a param with shape torch.Size([1000]) from checkpoint, the shape in current model is torch.Size([10]).
Continuing with pretrained weights only.
Model loaded on cuda
Available embedding dimensions: [8, 16, 32, 64, 128, 256, 512, 1024, 2048]
Output embedding dimension set to 8


## Working with MNIST

In [ ]:
def mnist_to_tensors(mnist_dataset):
    # Get all images and labels
    images = torch.stack([img for img, _ in mnist_dataset])
    labels = torch.tensor([label for _, label in mnist_dataset])
    
    # Add channel dimension if needed (MNIST usually has shape [N, H, W])
    if images.ndim == 3:
        images = images.unsqueeze(1)  # [N, 1, H, W]
    
    # Convert to 3-channel
    images = images.repeat(1, 3, 1, 1)
    
    return images, labels

mnist_images, mnist_labels = mnist_to_tensors(mnist_test)

In [75]:
print(mnist_labels)

tensor([7, 2, 1,  ..., 4, 5, 6])


In [88]:
mnist_embeds = model.extract_all_embeddings(mnist_images)
for dim, emb in mnist_embeds.items():
    print(f"Embedding dim {dim}: {emb.shape}")

Embedding dim 8: torch.Size([10000, 8])
Embedding dim 16: torch.Size([10000, 16])
Embedding dim 32: torch.Size([10000, 32])
Embedding dim 64: torch.Size([10000, 64])
Embedding dim 128: torch.Size([10000, 128])
Embedding dim 256: torch.Size([10000, 256])
Embedding dim 512: torch.Size([10000, 512])
Embedding dim 1024: torch.Size([10000, 1024])
Embedding dim 2048: torch.Size([10000, 2048])


In [101]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

def train_evaluate_logreg(embeddings_dict, labels, test_size=0.2, random_state=42):
    """
    Train and evaluate logistic regression classifiers for each embedding dimension.
    
    Args:
        embeddings_dict: Dictionary {dim: embeddings_tensor} where tensors are on CUDA
        labels: Tensor of corresponding labels
        test_size: Fraction of data to use for validation
        random_state: Random seed for reproducibility
        
    Returns:
        Dictionary of {dim: accuracy_score} for each dimension
    """
    # Convert labels to numpy (move to CPU if needed)
    labels_np = labels.cpu().numpy() if labels.is_cuda else labels.numpy()
    
    results = {}
    
    for dim, embeddings in embeddings_dict.items():
        # Convert embeddings to numpy (move to CPU if needed)
        emb_np = embeddings.cpu().numpy() if embeddings.is_cuda else embeddings.numpy()
        
        # Split into train/test
        X_train, X_test, y_train, y_test = train_test_split(
            emb_np, labels_np, 
            test_size=test_size, 
            random_state=random_state,
            stratify=labels_np  # Preserve class distribution
        )
        
        # Train logistic regression
        logreg = LogisticRegression(
            max_iter=1000,  # Increased for convergence
            solver='lbfgs',  # Good for multiclass
            random_state=random_state
        )
        logreg.fit(X_train, y_train)
        
        # Evaluate
        y_pred = logreg.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        
        results[dim] = accuracy
        
        print(f"Dimension {dim}: Test Accuracy = {accuracy:.4f}")
    
    return results

In [103]:
results = train_evaluate_logreg(mnist_embeds, mnist_labels, test_size=0.2, random_state=42)

Dimension 8: Test Accuracy = 0.1465
Dimension 16: Test Accuracy = 0.1690
Dimension 32: Test Accuracy = 0.2580
Dimension 64: Test Accuracy = 0.6055
Dimension 128: Test Accuracy = 0.8050
Dimension 256: Test Accuracy = 0.8705
Dimension 512: Test Accuracy = 0.9180
Dimension 1024: Test Accuracy = 0.9380
Dimension 2048: Test Accuracy = 0.9400


In [79]:
all_results = model.predict_with_all_embeddings(mnist_images, return_probabilities=True)
for dim, (probs, emb) in all_results.items():
    print(f"Dim {dim}: Predictions shape {probs.shape}, Embedding shape {emb.shape}")

Dim 8: Predictions shape torch.Size([10000, 10]), Embedding shape torch.Size([10000, 8])
Dim 16: Predictions shape torch.Size([10000, 10]), Embedding shape torch.Size([10000, 16])
Dim 32: Predictions shape torch.Size([10000, 10]), Embedding shape torch.Size([10000, 32])
Dim 64: Predictions shape torch.Size([10000, 10]), Embedding shape torch.Size([10000, 64])
Dim 128: Predictions shape torch.Size([10000, 10]), Embedding shape torch.Size([10000, 128])
Dim 256: Predictions shape torch.Size([10000, 10]), Embedding shape torch.Size([10000, 256])
Dim 512: Predictions shape torch.Size([10000, 10]), Embedding shape torch.Size([10000, 512])
Dim 1024: Predictions shape torch.Size([10000, 10]), Embedding shape torch.Size([10000, 1024])
Dim 2048: Predictions shape torch.Size([10000, 10]), Embedding shape torch.Size([10000, 2048])


## Working with cifar10

In [95]:
def cifar10_to_tensors(cifar10_dataset):
    # Get all images and labels
    images = torch.stack([img for img, _ in cifar10_dataset])
    labels = torch.tensor([label for _, label in cifar10_dataset])
    
    return images, labels

cifar10_images, cifar10_labels = cifar10_to_tensors(cifar10_test)

In [98]:
print(cifar10_images.shape)

torch.Size([10000, 3, 32, 32])


In [100]:
cifar10_embeds = model.extract_all_embeddings(cifar10_images)
for dim, emb in cifar10_embeds.items():
    print(f"Embedding dim {dim}: {emb.shape}")

Embedding dim 8: torch.Size([10000, 8])
Embedding dim 16: torch.Size([10000, 16])
Embedding dim 32: torch.Size([10000, 32])
Embedding dim 64: torch.Size([10000, 64])
Embedding dim 128: torch.Size([10000, 128])
Embedding dim 256: torch.Size([10000, 256])
Embedding dim 512: torch.Size([10000, 512])
Embedding dim 1024: torch.Size([10000, 1024])
Embedding dim 2048: torch.Size([10000, 2048])


In [102]:
cifar10_results = train_evaluate_logreg(cifar10_embeds, cifar10_labels, test_size=0.2, random_state=42)

Dimension 8: Test Accuracy = 0.1050
Dimension 16: Test Accuracy = 0.1150
Dimension 32: Test Accuracy = 0.1355
Dimension 64: Test Accuracy = 0.2405
Dimension 128: Test Accuracy = 0.3735
Dimension 256: Test Accuracy = 0.4430
Dimension 512: Test Accuracy = 0.4765
Dimension 1024: Test Accuracy = 0.4770
Dimension 2048: Test Accuracy = 0.4850


## Working with CelebFaces

In [7]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class CelebAAttributesDataset(Dataset):
    def __init__(self, img_dir, attr_path, image_size=224):
        """
        Args:
            img_dir: Directory with all the images
            attr_path: Path to list_attr_celeba.csv
            image_size: Target image size
        """
        self.img_dir = img_dir
        self.image_files = sorted(os.listdir(img_dir))
        
        # Load and preprocess attributes
        self.attributes = pd.read_csv(attr_path)
        self.attributes.replace(-1, 0, inplace=True)  # Replace -1 with 0
        self.attr_names = self.attributes.columns[1:]  # Skip image_id column
        
        # Image transformations
        self.transform = transforms.Compose([
            transforms.CenterCrop(178),  # Original celebA is 218x178
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet stats
                                std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # Load image
        img_path = os.path.join(self.img_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)
        
        # Get corresponding attributes
        img_id = self.image_files[idx]
        attrs = self.attributes[self.attributes['image_id'] == img_id].iloc[0,1:].values
        attrs = torch.tensor(attrs.astype('float32'))
        
        return image, attrs

image_folder = "/home/iliam/datasets/img_align_celeba/img_align_celeba"
attributes_folder = "/home/iliam/datasets/list_attr_celeba.csv"
dataset = CelebAAttributesDataset(img_dir=image_folder, attr_path=attributes_folder, image_size=224)
model = MRLLoader(weights_path=model_path, num_classes=2)

Initializing resnet50 model...


/home/iliam/MRL-env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/iliam/MRL-env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Applying BlurPool...
Loading weights from /home/iliam/dl-project/model/final_weights.pt...
Error loading weights: Error(s) in loading state_dict for ResNet:
	size mismatch for fc.nesting_classifier_0.weight: copying a param with shape torch.Size([1000, 2048]) from checkpoint, the shape in current model is torch.Size([2, 2048]).
	size mismatch for fc.nesting_classifier_0.bias: copying a param with shape torch.Size([1000]) from checkpoint, the shape in current model is torch.Size([2]).
Continuing with pretrained weights only.
Model loaded on cuda
Available embedding dimensions: [8, 16, 32, 64, 128, 256, 512, 1024, 2048]
Output embedding dimension set to 8


In [8]:
from tqdm import tqdm

def extract_celeba_embeddings(model, dataset, batch_size=32, device='cuda'):
    """
    Extract embeddings for CelebA images
    
    Args:
        model: Your embedding model
        dataset: CelebAAttributesDataset instance
        batch_size: Batch size for processing
        device: Device to use
        
    Returns:
        Tuple of (embeddings, attributes) where:
        - embeddings: Dictionary {dim: tensor_of_embeddings}
        - attributes: Tensor of all attributes
    """
    
    # Initialize storage
    available_dims = sorted(model.get_available_dimensions())
    embeddings = {dim: [] for dim in available_dims}
    all_attrs = []
    
    # Create dataloader
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    # Process batches
    for batch_imgs, batch_attrs in tqdm(dataloader, desc="Extracting embeddings"):
        batch_imgs = batch_imgs.to(device)
        
        batch_embeddings = model.extract_all_embeddings(batch_imgs)
            
        for dim in available_dims:
            embeddings[dim].append(batch_embeddings[dim].cpu())
            
        all_attrs.append(batch_attrs.cpu())
    
    # Concatenate results
    final_embeddings = {
        dim: torch.cat(emb_list, dim=0) 
        for dim, emb_list in embeddings.items()
    }
    final_attrs = torch.cat(all_attrs, dim=0)
    
    return final_embeddings, final_attrs

In [9]:
embeddings, attributes = extract_celeba_embeddings(
    model=model,
    dataset=dataset,
    batch_size=128
)

Extracting embeddings:  68%|██████▊   | 1071/1583 [08:13<05:12,  1.64it/s]

: 